When operating in individual elements, we observe big slowdowns in Modin.
TODO: The code below crosses rows in every iteration. Create another version that deals with one row at a time.

In [1]:
import pandas as pd
import numpy as np
import collections.abc
# Koalas needs these mappings
collections.Iterable = collections.abc.Iterable
collections.Callable = collections.abc.Callable
import databricks.koalas as ks
import utils

# It seems you need to set this option for performance reasons.
# See: https://github.com/databricks/koalas/issues/1769 (it seems the issue is not only related to apply())
ks.set_option('compute.default_index_type', 'distributed')

pandas_df = pd.read_csv('../datasets/yellow_tripdata_2015-01.csv')
koalas_df = ks.read_csv('../datasets/yellow_tripdata_2015-01.csv')

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/12/25 09:26:50 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/12/25 09:26:50 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [2]:
%%time_cell

# Taken from: erikbruin/nlp-on-student-writing-eda

koalas_df['discourse_nr'] = 1
counter = 1

# I had to reduce the number of iterations to 15 otherwise, it takes _A LOT_ of time for
# Modin.
for i in range(1, 5):
  if koalas_df.loc[i, 'VendorID'] == koalas_df.loc[i-1, 'VendorID']:
    counter += 1
    koalas_df.loc[i, 'discourse_nr'] = counter
  else:
    counter = 1
    koalas_df.loc[i, 'discourse_nr'] = counter

In [3]:
koalas_time = _TIMED_CELL
print(f"Koalas time: {koalas_time:.1f}s")

Koalas time: 15.5s


In [4]:
%%time_cell

# Taken from: erikbruin/nlp-on-student-writing-eda

pandas_df['discourse_nr'] = 1
counter = 1

for i in range(1, 5):
  if pandas_df.loc[i, 'VendorID'] == pandas_df.loc[i-1, 'VendorID']:
    counter += 1
    pandas_df.loc[i, 'discourse_nr'] = counter
  else:
    counter = 1
    pandas_df.loc[i, 'discourse_nr'] = counter

In [5]:
pandas_time = _TIMED_CELL
print(f"Pandas time: {pandas_time:.1f}s")

Pandas time: 0.1s


We have to use `to_pandas()`. Still fails with OOM but ok, let's hope.

In [7]:
assert (koalas_df['discourse_nr'].to_pandas() == pandas_df['discourse_nr']).all()

Exception in thread "refresh progress" java.lang.OutOfMemoryError: Java heap space
	at scala.Option.map(Option.scala:230)
	at org.apache.spark.status.AppStatusStore.activeStages(AppStatusStore.scala:164)
	at org.apache.spark.ui.ConsoleProgressBar.org$apache$spark$ui$ConsoleProgressBar$$refresh(ConsoleProgressBar.scala:64)
	at org.apache.spark.ui.ConsoleProgressBar$$anon$1.run(ConsoleProgressBar.scala:52)
	at java.base/java.util.TimerThread.mainLoop(Timer.java:556)
	at java.base/java.util.TimerThread.run(Timer.java:506)
Exception in thread "RemoteBlock-temp-file-clean-thread" java.lang.OutOfMemoryError: Java heap space
Exception in thread "Spark Context Cleaner" java.lang.OutOfMemoryError: Java heap space


22/12/25 09:28:08 ERROR Utils: uncaught error in thread Spark Context Cleaner, stopping SparkContext
java.lang.OutOfMemoryError: Java heap space
22/12/25 09:28:08 ERROR Utils: throw uncaught fatal error in thread Spark Context Cleaner
java.lang.OutOfMemoryError: Java heap space


Py4JJavaError: An error occurred while calling o794.collectToPython.
: java.lang.OutOfMemoryError: Java heap space
	at org.apache.spark.sql.execution.SparkPlan$$anon$1._next(SparkPlan.scala:391)
	at org.apache.spark.sql.execution.SparkPlan$$anon$1.getNext(SparkPlan.scala:402)
	at org.apache.spark.sql.execution.SparkPlan$$anon$1.getNext(SparkPlan.scala:388)
	at org.apache.spark.util.NextIterator.hasNext(NextIterator.scala:73)
	at scala.collection.Iterator.foreach(Iterator.scala:943)
	at scala.collection.Iterator.foreach$(Iterator.scala:943)
	at org.apache.spark.util.NextIterator.foreach(NextIterator.scala:21)
	at org.apache.spark.sql.execution.SparkPlan.$anonfun$executeCollect$1(SparkPlan.scala:425)
	at org.apache.spark.sql.execution.SparkPlan.$anonfun$executeCollect$1$adapted(SparkPlan.scala:424)
	at org.apache.spark.sql.execution.SparkPlan$$Lambda$3313/0x000000084130c840.apply(Unknown Source)
	at scala.collection.IndexedSeqOptimized.foreach(IndexedSeqOptimized.scala:36)
	at scala.coll